<a href="https://colab.research.google.com/github/iamemc/PD_01/blob/main/PD_202021_P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining / Prospeção de Dados

## Diogo Soares and Sara C. Madeira, 2020/21

# Project 1 - Pattern Mining

## Logistics 
**_Read Carefully_**

**Students should work in teams of 2 or 3 people**. 

**TASK 3 - Spring vs Summer Purchases** must be done only by groups of 3 people.

Individual projects might be allowed (with valid justification), but will not have better grades for this reason. 

The quality of the project will dictate its grade, not the number of people working.

**The project's solution should be uploaded in Moodle before the end of `March, 28th (23:59)`.** 

Students should **upload a `.zip` file** containing all the files necessary for project evaluation. 
Groups should be registered in [Moodle](https://moodle.ciencias.ulisboa.pt/mod/groupselect/view.php?id=139096) and the zip file should be identified as `PDnn.zip` where `nn` is the number of your group.

**It is mandatory to produce a Jupyter notebook containing code and text/images/tables/etc describing the solution and the results. Projects not delivered in this format will not be graded. You can use `PD_202021_P1.ipynb`as template. In your `.zip` folder you should also include an HTML version of your notebook with all the outputs** (File > Download as > HTML).

**Decisions should be justified and results should be critically discussed.** 

_Project solutions containing only code and outputs without discussions will achieve a maximum grade 10 out of 20._

## Dataset and Tools



In this project you will analyse data from an online Store collected over 4 months (April - July 2014). The folder `data` contains three files that you should use to obtain the dataset to be used in pattern mining. 

The file `store-buys.dat` comprises the buy events of the users over the items. It contains **318.444 sessions**. Each record/line in the file has the following fields (with this order): 

* **Session ID** - the id of the session. In one session there are one or many buying events. Could be represented as an integer number.
* **Timestamp** - the time when the buy occurred. Format of YYYY-MM-DDThh:mm:ss.SSSZ
* **Item ID** – the unique identifier of item that has been bought. Could be represented as an integer number. 
* **Price** – the price of the item. Could be represented as an integer number.
* **Quantity** – the quantity in this buying.  Could be represented as an integer number.

The file `store-clicks.dat` comprises the clicks of the users over the items. It contains **5.613.499 sessions**.  Each record/line in the file has the following fields (with this order):

* **Session ID** – the id of the session. In one session there are one or many clicks. Could be represented as an integer number.
* **Timestamp** – the time when the click occurred. Format of YYYY-MM-DDThh:mm:ss.SSSZ
* **Item ID** – the unique identifier of the item that has been clicked. Could be represented as an integer number.
* **Context** – the context of the click. The value "S" indicates a special offer, "0" indicates  a missing value, a number between 1 to 12 indicates a real category identifier,
any other number indicates a brand. E.g. if an item has been clicked in the context of a promotion or special offer then the value will be "S", if the context was a brand i.e BOSCH,
then the value will be an 8-10 digits number. If the item has been clicked under regular category, i.e. sport, then the value will be a number between 1 to 12. 
 
The file `products.csv` comprises the list of products sold by the online store. It contains **46.294 different products** associated with **123 different subcategories**. Each record/line in the file has the following fields:

* **Item ID** - the unique identifier of the item. Could be represented as an integer number. 
* **Product Categories** - the category and subcategories of the item. It is a string containing the category and subcategories of the item. Eg. `appliances.kitchen.juice`


In this project you should use [Python 3](https://www.python.org), [Jupyter Notebook](http://jupyter.org) and **[MLxtend](http://rasbt.github.io/mlxtend/)**. When using MLxtend, frequent patterns can either be discovered using `Apriori` and `FP-Growth`. **Choose the pattern mining algorithm to be used.** 


## Team Identification

**GROUP PD03**

Students:

* Eduardo Carvalho - nº55881
* Filipe Santos - nº55142
* Ivo Oliveira - nº50301

## 1. Mining Frequent Itemsets and Association Rules


In this first part of the project you should load and preprocess the dataset  in order to compute frequent itemsets and generate association rules considering all the sessions.

**In what follows keep the following question in mind and be creative!**

1. What are the most interesting products?
2. What are the most bought products?
3. Which products are bought together?
4. Can you find associations between the clicked products? 
5. Can you find associations highliting that when people buy a product/set of products also buy other product(s)?
6. Can you find associations highliting that when people click in a product/set of products also buy this product(s)?
7. Can you find relevant associated categories? 

### 1.1. Load and Preprocess Data

 **Product quantities were not be considered.**

##Limitations
*Due to a problem in the database quantity (QTY) column, the SessionID that corresponds to an equivalent ID in Buys, was thus considered as a purchase. Altough this skews the expected results, the fact some buys had quantities of 0 and other problems, this was a temporarly, yet accurate metric of products and their given clicks and purchases*


In [1]:
#!pip install mlxtend
!pip install mlxtend --upgrade

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import QuadMesh
from matplotlib.text import Text
from matplotlib.colors import LogNorm
import seaborn as sns
import tkinter as tkr
from mlxtend.preprocessing import  TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

Requirement already up-to-date: mlxtend in /usr/local/lib/python3.7/dist-packages (0.18.0)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
products=pd.read_csv('/content/drive/MyDrive/Datasets/projetoPD/products.csv',
                     header=None,
                     names=['ItemID','Category'],
                     dtype={'ItemID': np.int32, 'Category':str})
buys=pd.read_csv('/content/drive/MyDrive/Datasets/projetoPD/store-buys.dat', 
                 header=None,
                 names=['SessionID','TimeStamp','ItemID','Price','Qty'],
                 dtype={'SessionID':np.int32, 'TimeStamp':str, 'ItemID': np.int32, 
                        'Price':np.int16, 'Qty':'category'})
clicks=pd.read_csv('/content/drive/MyDrive/Datasets/projetoPD/store-clicks.dat', 
                   header=None,
                   nrows=1000000,
                   names=['SessionID','TimeStamp','ItemID','Context'],
                   dtype={'SessionID':np.int32, 'TimeStamp':str, 'ItemID': np.int32, 
                        'Context':'category'})

In [4]:
clicks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype   
---  ------     --------------    -----   
 0   SessionID  1000000 non-null  int32   
 1   TimeStamp  1000000 non-null  object  
 2   ItemID     1000000 non-null  int32   
 3   Context    1000000 non-null  category
dtypes: category(1), int32(2), object(1)
memory usage: 16.2+ MB


#Database Corrections
This code segment contains Database corrections of the following:
- products_no_duplicates - removed duplicates
- buys_upd - added product names & season

In [ ]:
"""
DATABASE CORRECTIONS
products_no_duplicates - removed duplicates
buys_upd - added product names & season
"""
buys = buys.drop(columns=['Qty'])
products_no_duplicates = products.drop_duplicates().reset_index(drop=True)

buys_upd =pd.merge(buys,products_no_duplicates)
clicks_upd= pd.merge(clicks,products_no_duplicates)

#ignore prices/quantities of 0
#buys_upd = buys_upd[buys_upd.Price>0]
#buys_upd = buys_upd[buys_upd.Qty>0]
#no click_upd ver context=0 (missing value)

product_name_buys=[]
for cat in buys_upd.Category:
  product_name_buys.append(cat.split('.')[-1].replace('_',' ').title())
product_name_clicks=[]
for cat in clicks_upd.Category:
  product_name_clicks.append(cat.split('.')[-1].replace('_',' ').title())
#product_name[:5]

buys_upd['ProductName']= product_name_buys
buys_upd = buys_upd.drop('Category', axis=1)
clicks_upd['ProductName']=product_name_clicks
clicks_upd=clicks_upd.drop('Category', axis=1)

dates_buys =[]
season_buys =[]
dates_clicks =[]
season_clicks =[]

for i in buys_upd["TimeStamp"]:
  dates_buys.append(i[:10])
  if i[5:7] == "04" or i[5:7] == "05":
     season_buys.append("Spring")
  elif i[5:7] == "06" or i[5:7] == "07":
    season_buys.append("Summer")
  else:
    season_buys.append("Other")

for i in clicks_upd["TimeStamp"]:
  dates_clicks.append(i[:10])
  if i[5:7] == "04" or i[5:7] == "05":
     season_clicks.append("Spring")
  elif i[5:7] == "06" or i[5:7] == "07":
    season_clicks.append("Summer")
  else:
    season_clicks.append("Other")

weekday_buys=[]
buys_upd["TimeStamp"] = pd.to_datetime(buys_upd["TimeStamp"])
buys_upd["Weekday_Num"]=buys_upd["TimeStamp"].dt.dayofweek 
for i in buys_upd["Weekday_Num"]:
  if i < 5: 
    weekday_buys.append("Weekday")
  else:
    weekday_buys.append("Weekend")

weekday_clicks=[]
clicks_upd["TimeStamp"] = pd.to_datetime(clicks_upd["TimeStamp"])
clicks_upd["Weekday_Num"]=clicks_upd["TimeStamp"].dt.dayofweek 
for i in clicks_upd["Weekday_Num"]:
  if i < 5: 
    weekday_clicks.append("Weekday")
  else :
    weekday_clicks.append("Weekend")
  
buys_upd = buys_upd.drop(columns=['TimeStamp'])
buys_upd.insert(1, "TimeStamp", dates_buys)
buys_upd["Season"] = season_buys
buys_upd["Weekday"] = weekday_buys
buys_upd.sort_values(by='TimeStamp')

clicks_upd = clicks_upd.drop(columns=['TimeStamp'])
clicks_upd.insert(1, "TimeStamp", dates_clicks)
clicks_upd["Season"] = season_clicks
clicks_upd["Weekday"] = weekday_clicks
clicks_upd.sort_values(by='TimeStamp')

In [ ]:
buys_upd=buys_upd.sort_values(by='SessionID').reset_index(drop=True)
buys_upd.head()

Existem 113 registos duplicados. Precisamos de os remover para avançar com a análise.

In [ ]:
buys_upd=buys_upd.drop_duplicates().reset_index(drop=True)
len(buys_upd)

In [ ]:
clicks_upd=clicks_upd.sort_values(by='SessionID').reset_index(drop=True)
clicks_upd.head()

In [ ]:
len(clicks)-len(clicks.drop_duplicates())

Existem 46 registos duplicados. Precisamos de os remover para avançar com a análise.

In [ ]:
clicks_upd=clicks_upd.drop_duplicates().reset_index(drop=True)

# Most interesting items?

## Most Clicked Products


In [ ]:
# What are the most interesting items? (most clicked)
most_clicks = clicks_upd.groupby(["ProductName"], as_index=False).count().sort_values(by="SessionID", ascending=False)
most_clicks = most_clicks[['ProductName','SessionID']].reset_index(drop=True)
most_clicks = most_clicks.rename(columns={'SessionID':'Clicks'})
most_clicks.index += 1
print("Table 1.")
most_clicks.head(10)

> *The top three most clicked products are Memory Chips, with a total of 71265, followed by Blenders with 56013 and lastly, Meat Grinders with 32859. Below the product purchases will be shown through a a bar graph.*

In [ ]:
#2. What are the most bought products?
quantities = buys_upd.groupby(["ProductName"], as_index=False).count().sort_values(by="SessionID", ascending=False)
quantities2 = quantities
quantities = quantities[['ProductName','SessionID']].reset_index(drop=True)
quantities = quantities.rename(columns={'SessionID':'SalesCount'})
quantities.index += 1 

quantities = quantities[quantities.SalesCount>5000]
most_bought = sns.catplot(x="ProductName", y="SalesCount", kind="bar", data = quantities)
most_bought.fig.suptitle('Figure 1. Number of Sales per Product')
for ax in most_bought.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(75)
most_bought.set(yticks=np.arange(0,60000,2500))
most_bought.fig.set_figheight(12.5)
most_bought.fig.set_figwidth(25)

> *The bar graph above, clearly shows the differences between product purchases. Interestingly products with more than 5 thousand sales don't exactly match to the most clicked items as seen in the previous table (Table 1.).
For example, more than 55 thousand Tennis products were sold, however the same product can't be found in the top ten of most clicked online.
The same happens in Lawn Mowers, with around 32.5 thousand sales.
Memory chips, the second most bought has 32.5 thousand sales, that's 45% of the number of respective product clicks of said product.*

In [ ]:
#prepare graph info
quantities2 = buys_upd
quantities2['SessionID'] = 1
daysofWeek={0:'Segunda-Feira',1:'Terça-Feira',2:'Quarta-Feira',3:'Quinta-Feira', 4:'Sexta-Feira',5:'Sábado',6:'Domingo'}
quantities2['Weekday_Num'] = quantities2['Weekday_Num'].map(daysofWeek)
quantities2 = quantities2.groupby(["Weekday_Num","ProductName"], as_index=False)['SessionID'].sum()

drop_column = quantities2.groupby(['ProductName'], as_index=False)['SessionID'].sum()
drop_column = drop_column.sort_values(by='SessionID', ascending=False)
drop_column = drop_column.tail(-20)
drop_column = drop_column['ProductName']

quantities2 = quantities2[~quantities2['ProductName'].isin(drop_column)]
quantities2 = quantities2.sort_values(by='SessionID', ascending=False).reset_index(drop=True)
quantities2 = quantities2.pivot_table(index='Weekday_Num', columns = 'ProductName')['SessionID']
quantities2 = quantities2.reindex(['Segunda-Feira','Terça-Feira', 'Quarta-Feira','Quinta-Feira', 'Sexta-Feira','Sábado','Domingo'])
# ----- get max & min values for graph
hm_vmax = quantities2.max()
hm_vmax = hm_vmax.max()
hm_vmin = quantities2.min()
hm_vmin = hm_vmin.min()
quantities2['Total'] = quantities2.sum(axis=1)

In [ ]:
#process data for heatmap graph
sns.set(font_scale = 1.4)
fig, ax = plt.subplots(figsize=(30,10))
plt.suptitle("Product clicks per day of the week")
sns.heatmap(quantities2, cmap='coolwarm', square=True, linewidths=1, annot=True, fmt="d", vmin=hm_vmin*2, vmax=hm_vmax/2.5)
quadmesh = ax.findobj(QuadMesh)[0] # get array of colors
facecolors = quadmesh.get_facecolors()
facecolors[np.arange(20,147,21)] = np.array([1,1,1,1]) # make colors of the last column white
quadmesh.set_facecolors = facecolors # set modified colors
for i in ax.findobj(Text): # set color of all text to black
    i.set_color('black')

> *Above is an heat map analysis of the 20 most sold products (sorted alphabetically) and their sales per day of the week. This visualization goes on pair with the previous bar graph, and in it we can see the sales dispersion clearly. On the rightmost column is the total number of sales per day.*

>> *Sunday is by far the most popular day, with a total of 109025 sales and with the cells with the higher numbers. Tennis products bought at Sundays which number 16183 are the combination with the higher count.*

>> *Besides Sundays, Saturdays and Mondays are the most popular days of the week to buy online with 62383 and 74115 purchases, respectively. On the opposite end, Tuesday is is the least popular day, with only 19855 purchases.*

>> *As shown before, Tennis products are the most popular, and this is true for any day of the week compared to other products within the same day.*









## 1.2. Compute Frequent Itemsets

* Compute frequent itemsets considering a minimum support of X%. 
* Present frequent itemsets organized by length (number of items). 
* List frequent 1-itemsets, 2-itemsets, 3-itemsets, etc with support of at least Y%.
* Change X and Y when it makes sense and discuss the results.

In [ ]:
#Setting up: Which products are bought together?
all_sessions_buy={}
for i in range(len(buys_upd)):
  all_sessions_buy[buys_upd.SessionID[i]]=[]

for i in range(len(buys_upd)):
  if buys_upd.ProductName[i] not in all_sessions_buy.get(buys_upd.SessionID[i]):
    all_sessions_buy[buys_upd.SessionID[i]].append(buys_upd.ProductName[i])

transactions_buy=list(all_sessions_buy.values())

In [ ]:
#Setting up: Which products are viewed together?
all_sessions_click={}
for i in range(len(clicks_upd)):
  all_sessions_click[clicks_upd.SessionID[i]]=[]

for i in range(len(clicks_upd)):
  if clicks_upd.ProductName[i] not in all_sessions_click.get(clicks_upd.SessionID[i]):
    all_sessions_click[clicks_upd.SessionID[i]].append(clicks_upd.ProductName[i])

transactions_click=list(all_sessions_click.values())

In [ ]:
#Compute binary databases
tr_enc = TransactionEncoder()

#buys
trans_array_buy = tr_enc.fit(transactions_buy).transform(transactions_buy)
binary_database_buy = pd.DataFrame(trans_array_buy, columns=tr_enc.columns_)
binary_database_buy.head(10)
#clicks
trans_array_click = tr_enc.fit(transactions_click).transform(transactions_click)
binary_database_click = pd.DataFrame(trans_array_click, columns=tr_enc.columns_)

In [ ]:
#Compute itemsets min_support = 1% apriori and association rules

#buys
frequent_itemsets_buy = apriori(binary_database_buy, min_support=0.01, use_colnames=True, low_memory=True)
frequent_itemsets_buy
#Rules grau 1
rules1_buy = association_rules(frequent_itemsets_buy, metric="confidence", min_threshold=0.01)
#rules1_buy
# add new column length
frequent_itemsets_buy['length'] = frequent_itemsets_buy['itemsets'].apply(lambda x: len(x))
# filter using pattern length = 2
frequent_2_itemsets_buy = frequent_itemsets_buy[frequent_itemsets_buy['length'] >= 2].reset_index(drop=True)
#frequent_2_itemsets_buy

In [ ]:
#clicks
frequent_itemsets_click = apriori(binary_database_click, min_support=0.01, use_colnames=True, low_memory=True)
frequent_itemsets_click
#Rules grau 1
rules1_click = association_rules(frequent_itemsets_click, metric="confidence", min_threshold=0.01)
#rules1_click
# add new column length
frequent_itemsets_click['length'] = frequent_itemsets_click['itemsets'].apply(lambda x: len(x))
# filter using pattern length = 2
frequent_2_itemsets_click = frequent_itemsets_click[frequent_itemsets_click['length'] >= 2].reset_index(drop=True)
#frequent_2_itemsets_click

In [ ]:
#FP-Growth é melhori que apriori
#buys
frequent_itemsets_fpg_buy=fpgrowth(binary_database_buy, min_support=0.01,use_colnames=True, low_memory=True)
frequent_itemsets_fpg_buy
# Generate association rules with confidence >= 90%

rules_buy = association_rules(frequent_itemsets_fpg_buy, metric = "confidence", min_threshold=0.01)
rules_buy
#clicks
frequent_itemsets_fpg_click=fpgrowth(binary_database_click, min_support=0.01,use_colnames=True, low_memory=True)
frequent_itemsets_fpg_click
# Generate association rules with confidence >= 90%

rules_click = association_rules(frequent_itemsets_fpg_click, metric = "confidence", min_threshold=0.01)
rules_click

### 1.3. Generate Association Rules from Frequent Itemsets

* Generate association rules with a choosed value (C) for minimum confidence. 
* Generate association rules with a choosed value (L) for minimum lift. 
* Generate association rules with both confidence >= C% and lift >= L.
* Change C and L when it makes sense and discuss the results.

### 1.4. Take a Look at Maximal Patterns: Compute Maximal Frequent Itemsets

### 1.5. Conclusions 

# 2. Week vs Weekend Purchases

In this part of the project you should analyse the consumption patterns during the week vs during the weekeed.

**In what follows keep the following question in mind and be creative!**

1. The most interesting products are the same during the week and the weekend? 
2. What are the most bought products during the week? And during the weekend?
3. There are differences between the sets of products bought during the week and the weekend?
4. Can you find different associations highliting that when people click in a product/set of products also buy this product(s) during the week vs the weekend?
5. Discuss the results obtained for the week sessions vs weekend sessions.

### 2.1. Load and Preprocess Data

 **Product quantities should not be considered.**

### 2.2. Compute Frequent Itemsets

* Compute frequent itemsets considering a minimum support of X%. 
* Present frequent itemsets organized by length (number of items). 
* List frequent 1-itemsets, 2-itemsets, 3-itemsets, etc with support of at least Y%.
* Change X and Y when it makes sense and discuss the results.

### 2.3. Generate Association Rules from Frequent Itemsets

* Generate association rules with a choosed value (C) for minimum confidence. 
* Generate association rules with a choosed value (L) for minimum lift. 
* Generate association rules with both confidence >= C% and lift >= L.
* Change C and L when it makes sense and discuss the results.

### 2.4. Conclusions 

# 3. [Only Groups of 3] Spring vs Summer Purchases

In this part of the project you should analyse the consumption patterns during the Spring months (April and May) vs Summer months (June and July).

**In what follows keep the following question in mind and be creative!**

1. The most interesting products are the same during the Spring and the Summer? 
2. What are the most bought products during the Spring? And during the Summer?
3. There are differences between the sets of products bought during the Spring and the Summer?
4. Can you find different associations highliting that when people click in a product/set of products also buy this product(s) during the Spring vs the Summer?
5. Discuss the results obtained for the Spring sessions vs Summer sessions.

### 3.1. Load and Preprocess Data

 **Product quantities should not be considered.**

### 3.2. Compute Frequent Itemsets

* Compute frequent itemsets considering a minimum support of X%. 
* Present frequent itemsets organized by length (number of items). 
* List frequent 1-itemsets, 2-itemsets, 3-itemsets, etc with support of at least Y%.
* Change X and Y when it makes sense and discuss the results.

### 3.3. Generate Association Rules from Frequent Itemsets

* Generate association rules with a choosed value (C) for minimum confidence. 
* Generate association rules with a choosed value (L) for minimum lift. 
* Generate association rules with both confidence >= C% and lift >= L.
* Change C and L when it makes sense and discuss the results.

### 3.4. Conclusions 

## 4. Conclusions
Draw some conclusions about this project work.